In [1]:
import pandas as pd

In [110]:
train = pd.read_csv('train_data_features_selected.csv')
test = pd.read_csv('test_data_features_selected.csv')
train["Credit_Score"].value_counts(normalize=True)

Credit_Score
Standard    0.545266
Poor        0.289002
Good        0.165732
Name: proportion, dtype: float64

In [111]:
test["Credit_Score"].value_counts(normalize=True)

Credit_Score
Standard    0.52995
Poor        0.29370
Good        0.17635
Name: proportion, dtype: float64

In [3]:
train_y = train["Credit_Score"]
train_x = train.drop(["Credit_Score"], axis=1)

test_y = test["Credit_Score"]
test_x = test.drop(["Credit_Score"], axis=1)

In [4]:
train_x

,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,...,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_!@9#%8,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,5.0,1,10,4,12,10,6.24,0.0,40.090839,28.916667,...,False,True,False,False,True,False,False,False,False,False
1,5.0,7,8,3,28,8,1.85,4.0,23.607892,26.750000,...,False,True,False,False,False,False,False,False,False,True
2,7.0,7,9,4,16,13,11.24,3.0,27.094189,0.000000,...,False,True,False,False,False,False,False,True,False,False
3,2.0,3,9,1,2,9,9.94,1.0,26.205042,24.250000,...,True,False,False,False,False,True,False,False,False,False
4,5.0,4,18,3,24,12,18.76,8.0,24.301922,24.083333,...,False,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47614,4.0,5,12,4,7,11,3.17,5.0,32.072697,32.166667,...,True,False,False,False,True,False,False,False,False,False
47615,2.0,5,7,0,-2,12,5.16,0.0,27.192416,20.333333,...,False,True,False,False,False,False,False,False,False,True
47616,5.0,5,3,3,12,9,5.55,1.0,35.131041,23.000000,...,False,True,False,False,False,False,True,False,False,False
47617,1.0,7,3,3,12,9,3.90,2.0,36.826610,16.666667,...,False,True,False,False,False,False,False,True,False,False


In [5]:
train_y

0            Good
1            Good
2        Standard
3            Good
4        Standard
           ...   
47614        Good
47615        Good
47616        Good
47617    Standard
47618        Poor
Name: Credit_Score, Length: 47619, dtype: object

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

cv_number = 3
penalties = ['l1', 'l2', 'none']
c_values = [1, 10, 50]
random_state = [42]
max_features = ['sqrt', 'log2']
n_estimators = [50, 100, 150]

methods_stats_df = pd.DataFrame(data=None, columns=['method', 'accuracy', 'scaler', 'params'])
methods_stats_df_index = 0

def append_method_stats(method, accuracy, scaler, params):
    global methods_stats_df_index
    methods_stats_df.loc[methods_stats_df_index] = [method, accuracy, scaler, params]
    methods_stats_df_index += 1

In [7]:
standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('logreg', LogisticRegression())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('logreg', LogisticRegression())])

param_grid = {
    'logreg__penalty': penalties,
    'logreg__C': c_values,
    'logreg__random_state': random_state
}

In [8]:
from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg', LogisticRegression())]),
             param_grid={'logreg__C': [1, 10, 50],
                         'logreg__penalty': ['l1', 'l2', 'none'],
                         'logreg__random_state': [42]},
             scoring='accuracy')

In [9]:
append_method_stats('LogisticRegression', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [10]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('logreg', LogisticRegression())]),
             param_grid={'logreg__C': [1, 10, 50],
                         'logreg__penalty': ['l1', 'l2', 'none'],
                         'logreg__random_state': [42]},
             scoring='accuracy')

In [11]:
append_method_stats('LogisticRegression', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [12]:
from sklearn.svm import SVC

standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('svc', SVC())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('svc', SVC())])

param_grid = {
    'svc__C': c_values,
    'svc__random_state': random_state
}

In [13]:
gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [1, 10, 50], 'svc__random_state': [42]},
             scoring='accuracy')

In [14]:
append_method_stats('SVC', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [15]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [1, 10, 50], 'svc__random_state': [42]},
             scoring='accuracy')

In [16]:
append_method_stats('SVC', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [17]:
from sklearn.neighbors import KNeighborsClassifier

standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('knn', KNeighborsClassifier())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('knn', KNeighborsClassifier())])

param_grid = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__p': [1, 2]
}

In [18]:
gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__algorithm': ['auto', 'ball_tree', 'kd_tree',
                                            'brute'],
                         'knn__n_neighbors': [3, 5, 7], 'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             scoring='accuracy')

In [19]:
append_method_stats('KNeighborsClassifier', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [20]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__algorithm': ['auto', 'ball_tree', 'kd_tree',
                                            'brute'],
                         'knn__n_neighbors': [3, 5, 7], 'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             scoring='accuracy')

In [21]:
append_method_stats('KNeighborsClassifier', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [22]:
from sklearn.naive_bayes import MultinomialNB

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('nb', MultinomialNB())])

param_grid = {
    'nb__alpha': [0, 0.5, 1.0, 1.5, 2.0],
    'nb__fit_prior': [True, False]
}

In [23]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\veindata\lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\bred7\anaconda3\envs\

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('nb', MultinomialNB())]),
             param_grid={'nb__alpha': [0, 0.5, 1.0, 1.5, 2.0],
                         'nb__fit_prior': [True, False]},
             scoring='accuracy')

In [24]:
append_method_stats('MultinomialNB', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [25]:
from sklearn.tree import DecisionTreeClassifier

standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('dt', DecisionTreeClassifier())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('dt', DecisionTreeClassifier())])

param_grid = {
    'dt__criterion': ['gini', 'entropy', 'log_loss'],
    'dt__splitter': ['best', 'random'],
    'dt__max_depth': [None, 10, 20],
    'dt__min_samples_split': [2, 4, 6],
    'dt__min_samples_leaf': [1, 2, 3],
    'dt__max_features': max_features,
    'dt__random_state': random_state
}

In [26]:
gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('dt', DecisionTreeClassifier())]),
             param_grid={'dt__criterion': ['gini', 'entropy', 'log_loss'],
                         'dt__max_depth': [None, 10, 20],
                         'dt__max_features': ['sqrt', 'log2'],
                         'dt__min_samples_leaf': [1, 2, 3],
                         'dt__min_samples_split': [2, 4, 6],
                         'dt__random_state': [42],
                         'dt__splitter': ['best', 'random']},
             scoring='accuracy')

In [27]:
append_method_stats('DecisionTreeClassifier', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [28]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('dt', DecisionTreeClassifier())]),
             param_grid={'dt__criterion': ['gini', 'entropy', 'log_loss'],
                         'dt__max_depth': [None, 10, 20],
                         'dt__max_features': ['sqrt', 'log2'],
                         'dt__min_samples_leaf': [1, 2, 3],
                         'dt__min_samples_split': [2, 4, 6],
                         'dt__random_state': [42],
                         'dt__splitter': ['best', 'random']},
             scoring='accuracy')

In [29]:
append_method_stats('DecisionTreeClassifier', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [33]:
methods_stats_df

,method,accuracy,scaler,params
0,LogisticRegression,0.676138,StandardScaler,"{'logreg__C': 1, 'logreg__penalty': 'none', 'l..."
1,LogisticRegression,0.676159,MinMaxScaler,"{'logreg__C': 50, 'logreg__penalty': 'l2', 'lo..."
2,SVC,0.724270,StandardScaler,"{'svc__C': 10, 'svc__random_state': 42}"
3,SVC,0.711166,MinMaxScaler,"{'svc__C': 10, 'svc__random_state': 42}"
4,KNeighborsClassifier,0.739831,StandardScaler,"{'knn__algorithm': 'auto', 'knn__n_neighbors':..."
5,KNeighborsClassifier,0.692308,MinMaxScaler,"{'knn__algorithm': 'auto', 'knn__n_neighbors':..."
6,MultinomialNB,0.636133,MinMaxScaler,"{'nb__alpha': 2.0, 'nb__fit_prior': True}"
7,DecisionTreeClassifier,0.688696,StandardScaler,"{'dt__criterion': 'gini', 'dt__max_depth': 10,..."
8,DecisionTreeClassifier,0.688675,MinMaxScaler,"{'dt__criterion': 'gini', 'dt__max_depth': 10,..."


In [34]:
from sklearn.ensemble import RandomForestClassifier

standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('rf', RandomForestClassifier())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('rf', RandomForestClassifier())])

param_grid = {
    'rf__criterion': ['gini', 'entropy'],
    'rf__max_features': max_features,
    'rf__random_state': random_state
}

In [35]:
gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf', RandomForestClassifier())]),
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_features': ['sqrt', 'log2'],
                         'rf__random_state': [42]},
             scoring='accuracy')

In [36]:
append_method_stats('RandomForestClassifier', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [37]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('rf', RandomForestClassifier())]),
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_features': ['sqrt', 'log2'],
                         'rf__random_state': [42]},
             scoring='accuracy')

In [38]:
append_method_stats('RandomForestClassifier', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

standard_scaler_pipeline = Pipeline([('scaler', StandardScaler()),
                                     ('gb', GradientBoostingClassifier())])

min_max_scaler_pipeline = Pipeline([('scaler', MinMaxScaler()),
                                    ('gb', GradientBoostingClassifier())])

param_grid = {
    'gb__learning_rate': [0.1, 0.5, 1.0],
    'gb__max_features': max_features,
    'gb__random_state': random_state
}

In [40]:
gridsearch = GridSearchCV(standard_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('gb', GradientBoostingClassifier())]),
             param_grid={'gb__learning_rate': [0.1, 0.5, 1.0],
                         'gb__max_features': ['sqrt', 'log2'],
                         'gb__random_state': [42]},
             scoring='accuracy')

In [41]:
append_method_stats('GradientBoostingClassifier', gridsearch.best_score_, 'StandardScaler', gridsearch.best_params_)

In [42]:
gridsearch = GridSearchCV(min_max_scaler_pipeline,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=cv_number)
gridsearch.fit(train_x, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('gb', GradientBoostingClassifier())]),
             param_grid={'gb__learning_rate': [0.1, 0.5, 1.0],
                         'gb__max_features': ['sqrt', 'log2'],
                         'gb__random_state': [42]},
             scoring='accuracy')

In [43]:
append_method_stats('GradientBoostingClassifier', gridsearch.best_score_, 'MinMaxScaler', gridsearch.best_params_)

In [44]:
methods_stats_df.to_csv('methods_stats.csv', index=False)

In [45]:
methods_stats_df

,method,accuracy,scaler,params
0,LogisticRegression,0.676138,StandardScaler,"{'logreg__C': 1, 'logreg__penalty': 'none', 'l..."
1,LogisticRegression,0.676159,MinMaxScaler,"{'logreg__C': 50, 'logreg__penalty': 'l2', 'lo..."
2,SVC,0.724270,StandardScaler,"{'svc__C': 10, 'svc__random_state': 42}"
3,SVC,0.711166,MinMaxScaler,"{'svc__C': 10, 'svc__random_state': 42}"
4,KNeighborsClassifier,0.739831,StandardScaler,"{'knn__algorithm': 'auto', 'knn__n_neighbors':..."
5,KNeighborsClassifier,0.692308,MinMaxScaler,"{'knn__algorithm': 'auto', 'knn__n_neighbors':..."
6,MultinomialNB,0.636133,MinMaxScaler,"{'nb__alpha': 2.0, 'nb__fit_prior': True}"
7,DecisionTreeClassifier,0.688696,StandardScaler,"{'dt__criterion': 'gini', 'dt__max_depth': 10,..."
8,DecisionTreeClassifier,0.688675,MinMaxScaler,"{'dt__criterion': 'gini', 'dt__max_depth': 10,..."
9,RandomForestClassifier,0.781936,StandardScaler,"{'rf__criterion': 'entropy', 'rf__max_features..."


In [84]:
# baseline:
test_y.value_counts()['Standard']/len(test_y)

0.52995

In [85]:
scaled_train_x = MinMaxScaler().fit_transform(train_x)
test_x_without_nan = test_x.fillna(0)
scaled_test_x = MinMaxScaler().fit_transform(test_x_without_nan)

In [86]:
random_forest = RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=random_state[0])
random_forest.fit(scaled_train_x, train_y)

RandomForestClassifier(criterion='entropy', random_state=42)

In [87]:
random_forest.score(scaled_test_x, test_y)

0.55015

In [118]:
knn_model = KNeighborsClassifier(n_neighbors=7, weights='distance', algorithm='auto', p=1)
scaled_train_x = StandardScaler().fit_transform(train_x)
scaled_test_x = StandardScaler().fit_transform(test_x_without_nan)
knn_model.fit(scaled_train_x, train_y)

KNeighborsClassifier(n_neighbors=7, p=1, weights='distance')

In [119]:
knn_model.score(scaled_test_x, test_y)

0.6233

In [90]:
gradient_boosting = GradientBoostingClassifier(learning_rate=0.5, max_features='sqrt', random_state=random_state[0])
gradient_boosting.fit(scaled_train_x, train_y)

GradientBoostingClassifier(learning_rate=0.5, max_features='sqrt',
                           random_state=42)

In [91]:
gradient_boosting.score(scaled_test_x, test_y)

0.5581

In [93]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier()
cross_val_score(bagging, scaled_train_x, train_y, cv=cv_number).mean()

0.7517167517167517

In [94]:
bagging.fit(scaled_train_x, train_y)
bagging.score(scaled_test_x, test_y)

0.57895

In [95]:
append_method_stats('BaggingClassifier', cross_val_score(bagging, scaled_train_x, train_y, cv=cv_number).mean(), 'StandardScaler', bagging.get_params())

In [120]:
scaled_train_x = StandardScaler().fit_transform(train_x)
scaled_test_x = StandardScaler().fit_transform(test_x_without_nan)

In [123]:
model = DecisionTreeClassifier(criterion='gini', max_features='sqrt', random_state=random_state[0])
model.fit(scaled_train_x, train_y)
model.score(scaled_test_x, test_y)

0.4888